In [471]:
# Biblioteki wejściowe
import pandas as pd
import numpy as np
import posixpath as os
import pathlib
from pathlib import Path
import pyodbc
import sys
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.offsets import MonthEnd
import IPython
from sklearn.linear_model import LinearRegression

pd.set_option("display.max_rows", None)
#pd.options.display.max_rows=50000
pd.options.display.max_colwidth=70
pd.options.display.precision=2
pd.options.display.max_columns=50
pd.options.display.float_format='{0:,.2f}'.format

In [472]:
# Pobiera dane wejściowe MAC

DATA = Path(r'C:\Users\DW\KOPALNIA\MIS_REPORT_2022\MONTHLY_REPORTS_2022\DATA')

t_konta_mis=pd.read_csv(DATA /'MIS_Konta_Analiza_2207.csv')
t_sprzedaz_fa=t=pd.read_excel(DATA /'FA_Sprzedaż_202207.xlsx')
t_sprzedaz_fa.fillna(0,inplace=True)
t_mis=t_konta_mis.groupby(['P0','P1','P2','M'],as_index=False)[['P_2022','W_2022']].sum()
t_ile=t_sprzedaz_fa.groupby(['M'],as_index=False)[['Ilość(t)']].sum()

In [473]:
t_ile.groupby(['M'],as_index=False)[['Ilość(t)']].sum()

,M,Ilość(t)
0,1,"32,569.80"
1,2,"68,381.08"
2,3,"112,294.47"
3,4,"101,692.85"
4,5,"107,406.95"
5,6,"112,499.35"
6,7,"119,069.70"


In [474]:
# Wykonanie sprzedaży

t_m=pd.DataFrame({'M':range(1,13)})
t_fr_ile=pd.merge(t_m,t_ile,on='M',how='left')
t_fr_ile.fillna(0,inplace=True)
t_fr_ile.rename(columns={'Ilość(t)':'WS_2022'},inplace=True)

# Forecast do końca roku

t_fr_ile['FS_2022']=t_fr_ile['WS_2022']
t_fr_ile['FS_2022']=t_fr_ile.apply(lambda x:120000 if x['M']==8 else x['FS_2022'],axis=1)
t_fr_ile['FS_2022']=t_fr_ile.apply(lambda x:120000 if x['M']==9 else x['FS_2022'],axis=1)
t_fr_ile['FS_2022']=t_fr_ile.apply(lambda x:100000 if x['M']==10 else x['FS_2022'],axis=1)
t_fr_ile['FS_2022']=t_fr_ile.apply(lambda x:75000 if x['M']==11 else x['FS_2022'],axis=1)
t_fr_ile['FS_2022']=t_fr_ile.apply(lambda x:75000 if x['M']==12 else x['FS_2022'],axis=1)

# Plan 2023

P_2023=900000 #------------------TUTAJ PLAN ROCZNY W TONACH !!!

t_fr_ile['PS_2023']=t_fr_ile['FS_2022']/t_fr_ile['FS_2022'].sum()*P_2023
t_forecast=pd.merge(t_mis,t_fr_ile,how='left',on='M')
t_forecast['F_2022']=t_forecast['W_2022']
t_forecast['P_2023']=0

In [475]:
# Plan sprzedaży WDT

temp=t_forecast[(t_forecast['M']<=7) & (t_forecast['P2'].str.startswith('Sprzedaż WDT'))]

x=temp['WS_2022'].to_numpy()
y=temp['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>7 and x['P2'].startswith('Sprzedaż WDT') else x['F_2022'],axis=1)

In [476]:
# Plan sprzedaży krajowej

P2='Sprzedaż krajowa'
temp=t_forecast[(t_forecast['M']<=7) & (t_forecast['P2'].str.startswith(P2))]

x=temp['WS_2022'].to_numpy()
y=temp['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>7 and x['P2'].startswith(P2) else x['F_2022'],axis=1)

In [477]:

# Amortyzacja

typ_a='Budowle'
t_am=t_forecast[(t_forecast['P1'].str.startswith('1. Amortyzacja')) & (t_forecast['P2'].str.startswith(typ_a))]
t_am_value=t_am.iloc[6,9]
t_forecast['F_2022']=t_forecast.apply(lambda x:t_am_value if x['M']>7 and x['P1'].startswith('1. Amortyzacja') and x['P2'].startswith(typ_a) else x['F_2022'],axis=1)

typ_a='Budynki i lokale'
t_am=t_forecast[(t_forecast['P1'].str.startswith('1. Amortyzacja')) & (t_forecast['P2'].str.startswith(typ_a))]
t_am_value=t_am.iloc[6,9]
t_forecast['F_2022']=t_forecast.apply(lambda x:t_am_value if x['M']>7 and x['P1'].startswith('1. Amortyzacja') and x['P2'].startswith(typ_a) else x['F_2022'],axis=1)

typ_a='Grunt i użytkowanie wieczyste'
t_am=t_forecast[(t_forecast['P1'].str.startswith('1. Amortyzacja')) & (t_forecast['P2'].str.startswith(typ_a))]
t_am_value=t_am.iloc[6,9]
t_forecast['F_2022']=t_forecast.apply(lambda x:t_am_value if x['M']>7 and x['P1'].startswith('1. Amortyzacja') and x['P2'].startswith(typ_a) else x['F_2022'],axis=1)

typ_a='Maszyny i urządzenia'
t_am=t_forecast[(t_forecast['P1'].str.startswith('1. Amortyzacja')) & (t_forecast['P2'].str.startswith(typ_a))]
t_am_value=t_am.iloc[6,9]
t_forecast['F_2022']=t_forecast.apply(lambda x:t_am_value if x['M']>7 and x['P1'].startswith('1. Amortyzacja') and x['P2'].startswith(typ_a) else x['F_2022'],axis=1)

typ_a='Pozostełe środki trwałe'
t_am=t_forecast[(t_forecast['P1'].str.startswith('1. Amortyzacja')) & (t_forecast['P2'].str.startswith(typ_a))]
t_am_value=t_am.iloc[6,9]
t_forecast['F_2022']=t_forecast.apply(lambda x:t_am_value if x['M']>7 and x['P1'].startswith('1. Amortyzacja') and x['P2'].startswith(typ_a) else x['F_2022'],axis=1)

typ_a='Środki transportu'
t_am=t_forecast[(t_forecast['P1'].str.startswith('1. Amortyzacja')) & (t_forecast['P2'].str.startswith(typ_a))]
t_am_value=t_am.iloc[6,9]
t_forecast['F_2022']=t_forecast.apply(lambda x:t_am_value if x['M']>7 and x['P1'].startswith('1. Amortyzacja') and x['P2'].startswith(typ_a) else x['F_2022'],axis=1)


In [478]:
# Koszt zmienny według ilości produkcji - Energia elektryczna

######################### FORECAST ######################################################################################################################################################

msc=7
P1='2. Zużycie materiałów i energii'
P2='Energia elektryczna'

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]

x=t_temp_w['WS_2022'].to_numpy()
y=t_temp_w['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

####################### PLAN ############################################################################################################################################################


In [479]:
# Koszt stały według średniej - materiały biurowe

######################### FORECAST ######################################################################################################################################

msc=7
P1='2. Zużycie materiałów i energii'
P2='Materiały biurowe'
cpi=0.1
savings=0.2

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [480]:
# Koszt zmienny według ilości produkcji - Materiały pozostałe

######################### FORECAST ######################################################################################################################################################

msc=7
P1='2. Zużycie materiałów i energii'
P2='Materiały pozostałe'

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]

x=t_temp_w['WS_2022'].to_numpy()
y=t_temp_w['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

####################### PLAN ############################################################################################################################################################

In [481]:
# Koszt zmienny według ilości produkcji - Materiały produkcyjne

######################### FORECAST ######################################################################################################################################################

msc=7
P1='2. Zużycie materiałów i energii'
P2='Materiały produkcyjne'

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]

x=t_temp_w['WS_2022'].to_numpy()
y=t_temp_w['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

####################### PLAN ############################################################################################################################################################

In [482]:
# Koszt zmienny według ilości produkcji - Materiały produkcyjne

######################### FORECAST ######################################################################################################################################################

msc=7
P1='2. Zużycie materiałów i energii'
P2='Paliwo'

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]

x=t_temp_w['WS_2022'].to_numpy()
y=t_temp_w['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

####################### PLAN ############################################################################################################################################################

In [483]:
# Koszt zmienny według ilości produkcji - Transport kolejowy

######################### FORECAST ######################################################################################################################################################

msc=7
P1='3.1. Usługi obce - transport'
P2='Transport kolejowy'

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]

x=t_temp_w['WS_2022'].to_numpy()
y=t_temp_w['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

####################### PLAN ############################################################################################################################################################

In [484]:
# Koszt zmienny według ilości produkcji - Transport samochodowy

######################### FORECAST ######################################################################################################################################################

msc=7
P1='3.1. Usługi obce - transport'
P2='Transport samochodowy'

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]

x=t_temp_w['WS_2022'].to_numpy()
y=t_temp_w['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

####################### PLAN ############################################################################################################################################################

In [485]:
# Koszt stały według średniej - badania i usługi pomiarowe

######################### FORECAST ######################################################################################################################################

msc=7
P1='3.2. Usługi obce - ekspl. złoża'
P2='Badania i usługi pomiarowe'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [486]:
# Koszt stały według średniej - dzierżawa gruntów

######################### FORECAST ######################################################################################################################################

msc=7
P1='3.2. Usługi obce - ekspl. złoża'
P2='Dzierżawa gruntów'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [487]:

# Koszt zmienny według ilości produkcji - Naprawy i remonty

######################### FORECAST ######################################################################################################################################################

msc=7
P1='3.2. Usługi obce - ekspl. złoża'
P2='Naprawy i remonty'

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]

x=t_temp_w['WS_2022'].to_numpy()
y=t_temp_w['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

####################### PLAN ############################################################################################################################################################


In [488]:
# Koszt zmienny według ilości produkcji - transport i załadunek wewnętrzny

######################### FORECAST ######################################################################################################################################################

msc=7
P1='3.2. Usługi obce - ekspl. złoża'
P2='Transport i zał. wewnętrzny'

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]

x=t_temp_w['WS_2022'].to_numpy()
y=t_temp_w['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

####################### PLAN ############################################################################################################################################################

In [489]:
# Koszt zmienny według ilości produkcji - roboty strzałowe

######################### FORECAST ######################################################################################################################################################

msc=7
P1='3.2. Usługi obce - ekspl. złoża'
P2='Usługi wiertniczo - strzałowe'

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]

x=t_temp_w['WS_2022'].to_numpy()
y=t_temp_w['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

####################### PLAN ############################################################################################################################################################

In [490]:
# Koszt zmienny według ilości produkcji - transport i obsługa bocznicy

######################### FORECAST ######################################################################################################################################################

msc=7
P1='3.3. Usługi obce - bocznica'
P2='Transport i obsługa bocznicy'

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]

x=t_temp_w['WS_2022'].to_numpy()
y=t_temp_w['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

####################### PLAN ############################################################################################################################################################

In [491]:
# Koszt stały według średniej - Wynajem maszyn i urządzeń

######################### FORECAST ######################################################################################################################################

msc=7
P1='3.2. Usługi obce - ekspl. złoża'
P2='Wynajem maszyn i urządzeń'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [492]:
# Koszt stały według średniej - Wynajem bocznicy

######################### FORECAST ######################################################################################################################################

msc=7
P1='3.3. Usługi obce - bocznica'
P2='Wynajem bocznicy'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [493]:
# Koszt stały według średniej - Leasing operacyjny

######################### FORECAST ######################################################################################################################################

msc=7
P1='3.4. Usługi obce - pozostałe'
P2='Leasing operacyjny'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [494]:
# Koszt stały według średniej - Monitoring TMS

######################### FORECAST ######################################################################################################################################

msc=7
P1='3.4. Usługi obce - pozostałe'
P2='Monitoring TMS'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [495]:
# Koszt stały według średniej - Ochrona

######################### FORECAST ######################################################################################################################################

msc=7
P1='3.4. Usługi obce - pozostałe'
P2='Ochrona'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [496]:
# Koszt stały według średniej - Poczta, telefon, internet

######################### FORECAST ######################################################################################################################################

msc=7
P1='3.4. Usługi obce - pozostałe'
P2='Poczta, telefon, internet'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [497]:
# Koszt stały według średniej - Pozostałe usługi

######################### FORECAST ######################################################################################################################################

msc=7
P1='3.4. Usługi obce - pozostałe'
P2='Pozostałe usługi'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [498]:
# Koszt stały według średniej - Telefon internet

######################### FORECAST ######################################################################################################################################

msc=7
P1='3.4. Usługi obce - pozostałe'
P2='Telefon, internet'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [499]:
# Koszt stały według średniej - usługi IT

######################### FORECAST ######################################################################################################################################

msc=7
P1='3.4. Usługi obce - pozostałe'
P2='Usługi IT'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [500]:
# Koszt zmienny według ilości produkcji - opłata eksploatacyjna

######################### FORECAST ######################################################################################################################################################

msc=7
P1='4. Podatki i opłaty'
P2='Opłata eksploatacyjna'

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]

x=t_temp_w['WS_2022'].to_numpy()
y=t_temp_w['W_2022'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)

t_forecast['F_2022']=t_forecast.apply(lambda x:x['FS_2022']*model.coef_[0]+model.intercept_ if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

####################### PLAN ############################################################################################################################################################

In [501]:
# Koszt stały według średniej - opłaty środowiskowe

######################### FORECAST ######################################################################################################################################

msc=7
P1='4. Podatki i opłaty'
P2='Opłaty środowiskowe'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [502]:
# Koszt stały według średniej - PFRON

######################### FORECAST ######################################################################################################################################

msc=7
P1='4. Podatki i opłaty'
P2='PFRON'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [503]:
# Koszt stały według średniej - podatek leśny

######################### FORECAST ######################################################################################################################################

msc=7
P1='4. Podatki i opłaty'
P2='Podatek leśny'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [504]:
# Koszt stały według średniej - podatek od nieruchomości

######################### FORECAST ######################################################################################################################################

msc=7
P1='4. Podatki i opłaty'
P2='Podatek od nieruchomości'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [505]:
# Koszt stały według średniej - podatek rolny

######################### FORECAST ######################################################################################################################################

msc=7
P1='4. Podatki i opłaty'
P2='Podatek rolny'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [506]:
# Koszt stały według średniej - pozostałe opłaty

######################### FORECAST ######################################################################################################################################

msc=7
P1='4. Podatki i opłaty'
P2='Pozostałe opłaty'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [507]:
# Koszt stały według średniej - Uzytkowanie gruntów leśnych

######################### FORECAST ######################################################################################################################################

msc=7
P1='4. Podatki i opłaty'
P2='Uzytkowanie gruntów leśnych'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [508]:
# Koszt stały według średniej - Użytkowanie wieczyste

######################### FORECAST ######################################################################################################################################

msc=7
P1='4. Podatki i opłaty'
P2='Użytkowanie wieczyste'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w['W_2022'].mean()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [509]:
# Koszt stały według czerwca - Rada Nadzorcza

######################### FORECAST ######################################################################################################################################

msc=7
P1='5. Wynagrodzenia'
P2='Rada nadzorcza'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w.iloc[5,9]

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [510]:
# Koszt stały według czerwca - Umowy o pracę

######################### FORECAST ######################################################################################################################################

msc=7
P1='5. Wynagrodzenia'
P2='Umowy o pracę'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w.iloc[5,9]

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [511]:
# Koszt stały według czerwca - Umowy zlecenia

######################### FORECAST ######################################################################################################################################

msc=7
P1='5. Wynagrodzenia'
P2='Umowy zlecenia'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w.iloc[5,9]

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [512]:
# Koszt stały według czerwca - Zarząd i Prokura

######################### FORECAST ######################################################################################################################################

msc=7
P1='5. Wynagrodzenia'
P2='Zarząd i Prokura'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w.iloc[6,9]

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [513]:
# Koszt stały według czerwca - Fundusz socjalny

######################### FORECAST ######################################################################################################################################

msc=7
P1='6. Ubezpieczenia społeczne'
P2='Fundusz socjalny'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w.iloc[6,9]

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [514]:
# Koszt stały według czerwca - PPK

######################### FORECAST ######################################################################################################################################

msc=7
P1='6. Ubezpieczenia społeczne'
P2='PPK'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w.iloc[5,9]

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [515]:
# Koszt stały według czerwca - Składki ZUS

######################### FORECAST ######################################################################################################################################

msc=7
P1='6. Ubezpieczenia społeczne'
P2='Składki ZUS'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w.iloc[5,9]

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [516]:
# Koszt stały według czerwca - Szkolenia

######################### FORECAST ######################################################################################################################################

msc=7
P1='6. Ubezpieczenia społeczne'
P2='Szkolenia'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w.iloc[5,9]

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [517]:
# Koszt stały według czerwca - Świadczenia na rzecz pracowników

######################### FORECAST ######################################################################################################################################

msc=7
P1='6. Ubezpieczenia społeczne'
P2='Świadczenia na rzecz pracowników'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=t_temp_w.iloc[5,9]

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [518]:
t_lista=t_forecast[t_forecast['P1'].str.startswith('7')]
t_lista=t_lista['P2'].drop_duplicates()
# Koszt stały według czerwca - lista

######################### FORECAST ######################################################################################################################################

for x in t_lista:
    msc=7
    P1='7. Pozostałe koszty rodzajowe'
    P2=x
    cpi=0
    savings=0

    t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
    v_mean=t_temp_w.iloc[5,9]

    t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [528]:
# Wynik na pozostałej działalności - przeksięgowanie rekultywacji

######################### FORECAST ######################################################################################################################################

msc=7
P1='2. Inne koszty operacyjne'
P2='Rekultywacja'
cpi=0
savings=0

t_temp_w=t_forecast[(t_forecast['P1'].str.startswith(P1)) & (t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
v_mean=-t_temp_w['W_2022'].sum()

t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']==12 and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [531]:
t_lista=t_forecast[t_forecast['P0'].str.startswith('E')]
t_lista=t_lista['P2'].drop_duplicates()
# Przychody finansowe - lista

######################### FORECAST ######################################################################################################################################

for x in t_lista:
    msc=7
    P2=x
    cpi=0
    savings=0

    t_temp_w=t_forecast[(t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
    v_mean=t_temp_w.iloc[6,9]

    t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and x['P1'].startswith(P1) and x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [539]:
t_lista=t_forecast[t_forecast['P0'].str.startswith('F')]
t_lista=t_lista['P2'].drop_duplicates()
# Koszty finansowe - lista

######################### FORECAST ######################################################################################################################################

for x in t_lista:
    msc=7
    P2=x
    cpi=0
    savings=0

    t_temp_w=t_forecast[(t_forecast['P2'].str.startswith(P2)) & (t_forecast['M']<=msc)]
    v_mean=t_temp_w.iloc[5,9]

    t_forecast['F_2022']=t_forecast.apply(lambda x:(v_mean*(1+cpi/12)**(x['M']-msc))*(1-savings) if x['M']>msc and  x['P2'].startswith(P2) else x['F_2022'],axis=1)

######################### PLAN 2023 #######################################################################################################################################

In [541]:
t_forecast.to_clipboard(decimal=',',excel=True,index=False)

In [540]:
t_forecast[(t_forecast['P0'].str.startswith('E'))|t_forecast['P0'].str.startswith('F')]

,P0,P1,P2,M,P_2022,W_2022,WS_2022,FS_2022,PS_2023,F_2022,P_2023
1020,E. Przychody finansowe,1. Odsetki,Odsetki,1,0.00,0.00,"32,569.80","32,569.80","25,625.02",0.00,0
1021,E. Przychody finansowe,1. Odsetki,Odsetki,2,0.00,0.00,"68,381.08","68,381.08","53,800.34",0.00,0
1022,E. Przychody finansowe,1. Odsetki,Odsetki,3,0.00,0.00,"112,294.47","112,294.47","88,350.18",0.00,0
1023,E. Przychody finansowe,1. Odsetki,Odsetki,4,0.00,0.00,"101,692.85","101,692.85","80,009.12",0.00,0
1024,E. Przychody finansowe,1. Odsetki,Odsetki,5,0.00,0.00,"107,406.95","107,406.95","84,504.81",0.00,0
1025,E. Przychody finansowe,1. Odsetki,Odsetki,6,0.00,0.00,"112,499.35","112,499.35","88,511.37",0.00,0
1026,E. Przychody finansowe,1. Odsetki,Odsetki,7,0.00,0.00,"119,069.70","119,069.70","93,680.74",0.00,0
1027,E. Przychody finansowe,1. Odsetki,Odsetki,8,0.00,0.00,0.00,"120,000.00","94,412.68",0.00,0
1028,E. Przychody finansowe,1. Odsetki,Odsetki,9,0.00,0.00,0.00,"120,000.00","94,412.68",0.00,0
1029,E. Przychody finansowe,1. Odsetki,Odsetki,10,0.00,0.00,0.00,"100,000.00","78,677.23",0.00,0


In [521]:
t_forecast.to_clipboard(decimal=',',excel=True,index=False)